# Setup

Initial module setup.

In [14]:
import typing
import time
import statistics
import numpy.typing as np_types
import pandas as pd

from sklearn.svm import LinearSVC
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

# Constants

In [15]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10
RANDOM_SEED = 42

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [16]:
class ARSVMBuilder(model.ModelBuilder[LinearSVC]):
    def create_classifier(self) -> LinearSVC:
        return LinearSVC(
            random_state=RANDOM_SEED,
            dual=True,
            max_iter=2000
        )
    
    def train_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [17]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [18]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [19]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.ARFeatureExtractor({'lags': 25})
])

## Data Processor

In [20]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [21]:
ar_svm_builder = ARSVMBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = ar_svm_builder.train(K_FOLDS)

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/experiment/.local/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarni

In [22]:
subject_score_rows = []
for subject in results.training_statistics:
    subject_score_rows.append([
        subject,
        results.training_statistics[subject].average_score,
        results.training_statistics[subject].training_duration
    ])
subject_scores = pd.DataFrame(subject_score_rows, columns=['Subject', 'Average Score', 'Training Duration'])
subject_scores

,Subject,Average Score,Training Duration
0,S17,0.551836,7.302945
1,S12,0.770772,5.542626
2,S18,0.784522,4.797816
3,S02,0.687068,5.496783
4,S01,0.622160,6.302620
5,S05,0.658534,6.345356
6,S07,0.810247,6.737638
7,S19,0.858457,2.655695
8,S08,0.722654,5.750280
9,S20,0.710123,6.946935


In [23]:
subject_rates_rows = []
for subject in results.training_statistics:
    subject_rates_rows.append([
        subject,
        results.training_statistics[subject].average_false_accept_rate,
        results.training_statistics[subject].average_false_reject_rate
    ])
subject_rates = pd.DataFrame(subject_rates_rows, columns=['Subject', 'Average FAR', 'Average FRR'])
subject_rates

,Subject,Average FAR,Average FRR
0,S17,0.429272,0.825000
1,S12,0.205998,0.700000
2,S18,0.205827,0.400000
3,S02,0.304443,0.483333
4,S01,0.367515,0.600000
5,S05,0.324288,0.547619
6,S07,0.153947,0.900000
7,S19,0.148889,0.000000
8,S08,0.265755,0.525000
9,S20,0.261842,0.841667


In [24]:
global_stats = pd.DataFrame(
    [[results.global_average_score, results.global_average_far_rate, 
      results.global_average_frr_rate, results.global_average_time]],
    columns=['Global Average Score', 'Global Average FAR', 'Global Average FRR', 'Global Average Time']
)
global_stats

,Global Average Score,Global Average FAR,Global Average FRR,Global Average Time
0,0.693519,0.569048,0.292917,5.929202


# Simulated Execution

In [25]:
def simulate_model_executions(models: typing.Dict[str, LinearSVC], 
                              test_downloader: data.AuditoryDataDownloader, 
                              test_reader: data.AuditoryDataReader,
                              test_processor: processor.DataProcessor,
                              tests = 10) -> typing.Tuple[int, float]:
    data_path = test_downloader.retrieve()
    test_data = test_reader.format_data(data_path)
    for subject, data_samples in test_data.items():
        test_data[subject] = test_processor.process(data_samples)
    predict_timings = []
    for i in range(tests):
        for subject, prepared_data in test_data.items():
            test_model = models[subject]
            start_predict = time.perf_counter()
            test_model.predict(prepared_data)
            end_predict = time.perf_counter()
            predict_timings.append(end_predict - start_predict)
    return len(predict_timings), statistics.mean(predict_timings)

In [26]:
executions, average_predict_time = simulate_model_executions(results.models, downloader, reader, data_processor, tests=1)
print(f"Executions: {executions}")
print(f"Overall average predict time: {average_predict_time} seconds")

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..